In [42]:
from transformers import AutoModel, AutoTokenizer, \
BertForSequenceClassification, GPT2LMHeadModel

In [43]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [44]:
emb = model.bert.embeddings.word_embeddings

In [45]:

# freeze training for index 72 of the embedding layer
emb.weight[72].requires_grad = False
'''
RuntimeError: you can only change requires_grad flags of leaf variables.
If you want to use a computed variable in a subgraph that doesn't require differentiation use var_no_grad = var.detach()
可能需要用另一个办法, 记住initial value然后每次backpropagation之后再赋值回去
'''

# # make a copy of the initial value
# initial_value = emb.weight[72].clone()
# # print(emb.weight[72])
# # print(emb.weight[72].requires_grad)
# initial_value = initial_value.detach()
# initial_value.shape



RuntimeError: you can only change requires_grad flags of leaf variables. If you want to use a computed variable in a subgraph that doesn't require differentiation use var_no_grad = var.detach().

In [ ]:
# train the model on SST-2
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

dataset = load_dataset('glue', 'sst2')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding='max_length', truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
small_train_dataset = tokenized_dataset['train'].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_dataset['validation'].shuffle(seed=42).select(range(100))

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=small_train_dataset,         # training dataset
    eval_dataset=small_eval_dataset,           # evaluation dataset
)

trainer.train()

# change training loop to freeze the embedding layer
# https://huggingface.co/transformers/main_classes/trainer.html#training-loop
# https://huggingface.co/transformers/main_classes/trainer.html#trainercallback
# https://huggingface.co/transformers/main_classes/callback.html#transformers.TrainerCallback
# https://huggingface.co/transformers/main_classes/callback.html#transformers.TrainerState
# https://huggingface.co/transformers/main_classes/callback.html#transformers.TrainerControl
# https://huggingface.co/transformers/main_classes/callback.html#transformers.TrainerState

from transformers import TrainerCallback, TrainerControl, TrainerState

class FreezeEmbeddingCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        # print(state)
        # print(control)
        # print(state.model.bert.embeddings.word_embeddings.weight[72])
        # print(state.model.bert.embeddings.word_embeddings.weight[72].requires_grad)
        # print(initial_value)
        # print(initial_value.shape)
        # print(state.model.bert.embeddings.word_embeddings.weight[72].shape)
        state.model.bert.embeddings.word_embeddings.weight[72] = initial_value
        # print(state.model.bert.embeddings.word_embeddings.weight[72])
        # print(state.model.bert.embeddings.word_embeddings.weight[72].requires_grad)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=small_train_dataset,         # training dataset
    eval_dataset=small_eval_dataset,           # evaluation dataset
    callbacks=[FreezeEmbeddingCallback()]
)

trainer.train()


